# MVP phenotype terms, round 2, mapping terms

In [1]:
%load_ext autoreload
%autoreload 2

%load_ext lab_black

In [2]:
import sys
from pathlib import Path

In [3]:
_pwd = Path(".").resolve()
print(_pwd)
sys.path.append(str(_pwd))

/data/ik18445_cache/projects/phenotype-mapping/analysis/notebooks/mvp_round_2


In [4]:
from typing import List
import re
import json
import math

import pandas as pd
import janitor
import numpy as np
from pydash import py_

import spacy
import scispacy

import ray

from common_funcs import utils
from analysis_funcs import paths
import mvp_funcs, mvp_types

In [5]:
proj_root = utils.find_project_root("docker-compose.yml")
assert proj_root.exists(), proj_root

data_path = proj_root / "data"

input_path = data_path / "output" / "mvp-mapping-round-2"
assert input_path.exists(), input_path

# Load in 

In [6]:
input_file = input_path / "mvp-encode.json"
assert input_file.exists(), input_file

with input_file.open() as f:
    dat_encode = json.load(f)

print(len(dat_encode))

5404


# Util funcs

# sample test

In [7]:
sample = dat_encode[:15]
for idx, _ in enumerate(sample):
    print(idx, _["trait_id"], _["trait_term_encode"]["term"])

0 UKBB+MVP-8-00 Intestinal infection
1 UKBB+MVP-85-01 Bacterial enteritis
2 MVP-851-02 Intestinal e.coli
3 UKBB+MVP-852-03 Intestinal infection due to C. difficile
4 UKBB+MVP-86-04 Viral Enteritis
5 MVP-87-05 Intestinal infection due to protozoa
6 UKBB+MVP-10-06 Tuberculosis
7 MVP-31-07 Diseases due to other mycobacteria
8 MVP-311-08 Leprosy
9 UKBB+MVP-38-09 Septicemia
10 UKBB+MVP-381-10 Gram negative septicemia
11 UKBB+MVP-382-11 Gram positive septicemia
12 MVP-383-12 Bacteremia
13 UKBB+MVP-41-13 Bacterial infection
14 UKBB+MVP-411-14 Staphylococcus infections


In [8]:
sample_mapping = [mvp_funcs.get_mapping(_) for _ in sample]

output_file = input_file.parent / "mvp-mapping-sample.json"
with output_file.open("w") as f:
    json.dump(sample_mapping, f)

# actual processing

In [11]:
NUM_WORKERS = 8

if ray.is_initialized():
    ray.shutdown()
ray.init(num_cpus=NUM_WORKERS)

2022-11-07 11:42:08,718	INFO worker.py:1518 -- Started a local Ray instance.


Python version:,3.8.13
Ray version:,2.0.1


In [ ]:
ray.shutdown()

# post processing diagnostics

# wrap up